# Assignment Chapter 3 - COMPUTER VISION [Case #3]
Startup Campus, Indonesia - `Artificial Intelligence (AI)` (Batch 7)
* Dataset: Any YouTube videos
* Libraries: PyTorch, Numpy, OpenCV2
* Objective: Real-time Object Detection using CNN-based Pre-trained Models

`PERSYARATAN` Semua modul (termasuk versi yang sesuai) sudah di-install dengan benar.
<br>`CARA PENGERJAAN` Lengkapi baris kode yang ditandai dengan **#TODO**.
<br>`TARGET PORTFOLIO` Peserta mampu mengimplementasikan model PyTorch untuk mendeteksi objek secara *real-time*.

### Install additional library

In [ ]:
!pip install cap-from-youtube

In [ ]:
# nambahin sendiri (nyoba)
!pip install -U ultralytics

### Import Libraries

In [ ]:
import torch
import numpy as np
import cv2

from cap_from_youtube import cap_from_youtube
from time import time
from datetime import datetime as dt

### User-defined Class

In [ ]:
class ObjectDetection:
    def __init__(self, url, out_file="{}_video.avi".format(dt.now().strftime("%Y%m%d_%H%M%S"))):
        """
        Initializes the class with youtube url and output file.
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """

        self._URL = url
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """

        return cap_from_youtube(self._URL)

    def load_model(self):
        """
        Loads the model from pytorch hub.
        :return: Trained Pytorch model.
        """

        # TODO: Panggil model ultralytics/yolov5
        # Lihat caranya di https://pytorch.org/hub/ultralytics_yolov5/#load-from-pytorch-hub
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """

        self.model.to(self.device)

        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].cpu().numpy(), results.xyxyn[0][:, :-1].cpu().numpy()
        return labels, cord

    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """

        return self.classes[int(x)]

    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """

        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """

        player = self.get_video_from_url()
        assert player.isOpened()

        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))

        for i in range(1, 300):
            start_time = time()
            ret, frame = player.read()

            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()

            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")
            out.write(frame)

### IMPORTANT: Activate your GPU

- Di Google Colab, klik **Runtime > Change runtime time**, lalu silakan pilih **T4 GPU**.

### Start Object Detection

In [ ]:
if __name__ == "__main__":
    # Pastikan CUDA enable: TRUE
    print("CUDA enable: {}".format(torch.cuda.is_available()))

    # TODO: Isi parameter dengan URL YouTube yang tersedia (secara bergantian):
    # 1. Crowded place: https://www.youtube.com/watch?v=dwD1n7N7EAg
    # 2. Solar system: https://www.youtube.com/watch?v=g2KmtA97HxY
    # 3. Road traffic: https://www.youtube.com/watch?v=wqctLW0Hb_0

    run_model = ObjectDetection(url="https://www.youtube.com/watch?v=g2KmtA97HxY")
    run_model()

CUDA enable: False


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-3 Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


[youtube] Extracting URL: https://www.youtube.com/watch?v=g2KmtA97HxY
[youtube] g2KmtA97HxY: Downloading webpage
[youtube] g2KmtA97HxY: Downloading ios player API JSON
[youtube] g2KmtA97HxY: Downloading mweb player API JSON
[youtube] g2KmtA97HxY: Downloading m3u8 information


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frames Per Second : 2.3980815347721824


...

Frames Per Second : 2.34192037470726


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frames Per Second : 2.277904328018223


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frames Per Second : 2.1881838074398248


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# [ PERTANYAAN ]
# TODO: Apa perbedaan "image classification" dan "object detection"?

[ ANSWER HERE ]

Image Classification:
-	Mengidentifikasi isi/konten utama dalam gambar.
-	Memberikan satu label untuk keseluruhan gambar.
Contoh: Mengklasifikasi gambar sebagai "kucing" atau "anjing".
-	Kompleksitas lebih sederhana karena komputasi yang ringan dan training yang lebih mudah.
-	Outputnya hanya menghasilkan label kelas dan berbentuk single output per gambar.
-	Arsitektur model CNN sederhana dengan menggunakan ResNet, VGG, ataupun Inception.


Object Detection:
-	Menemukan dan mengidentifikasi multiple objek dalam gambar.
-	Memberikan lokasi spesifik setiap objek dengan bounding box.
Contoh: Mendeteksi posisi semua kucing dan anjing dalam satu gambar.
-	Kompleksitas lebih kompleks karena komputasi yang lebih berat dan training yang lebih rumit.
-	Outputnya menghasilkan label kelas dan Lokasi objek yang berbentuk multiple output per gambar.
-	Arsitektur model kompleks dengan menggunakan YOLO, Faster R-CNN, ataupun SSD.


In [ ]:
# [ PERTANYAAN ]
# TODO: Di video mana YOLOv5 memiliki akurasi deteksi terburuk? Mengapa?

[ ANSWER HERE ]

YOLOv5 yang memiliki akurasi terburuk ada pada video ke-2 tata surya (Solar system) karena pada dasarnya YOLOv5 dilatih menggunakan dataset yang berisi gambar objek dunia nyata seperti manusia, kendaraan, hewan, dan benda sehari-hari. Video tata surya memperlihatkan objek seperti planet, bintang, dan elemen ruang angkasa lainnya yang tidak ada dalam dataset pelatihan YOLOv5. Sehingga model tidak memiliki pemahaman visual tentang bentuk atau karakteristik objek-objek tersebut, maka pada akhirnya sulit untuk mengenali dan tidak dapat memberikan deteksi yang akurat karena objek-objek ini tidak ada dalam kategori yang diketahui.

### Scoring
Total `#TODO` = 4
<br>Checklist:

- [x] Panggil model ultralytics/yolov5
- [x] Isi parameter dengan URL YouTube yang tersedia
- [x] [ PERTANYAAN ] Apa perbedaan "image classification" dan "object detection"?
- [x] [ PERTANYAAN ] Di video mana YOLOv5 memiliki akurasi deteksi terburuk? Mengapa?

### Additional readings
* N/A

### Copyright © 2024 Startup Campus, Indonesia
* Prepared by **Nicholas Dominic, M.Kom.** [(profile)](https://linkedin.com/in/nicholas-dominic)
* You may **NOT** use this file except there is written permission from PT. Kampus Merdeka Belajar (Startup Campus).
* Please address your questions to mentors.